#---------- OCR Part ----------

#OCR & img2str & Augment img

In [1]:
!pip install  -q pdfplumber
!pip install  -q pytesseract
!pip install  -q pdf2image
!sudo apt update -q
!sudo apt install -y -q poppler-utils
!sudo apt install  -q tesseract-ocr
!apt-get install -y  -q tesseract-ocr-tha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 41.9 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,918 kB]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://pp

In [2]:
import pdfplumber
from pdf2image import convert_from_path
from tqdm import tqdm
import torch
import pytesseract
import cv2
import numpy as np

In [ ]:
# pdf_path = "/content/167631(1).pdf"
pdf_path = "/content/167631(1)-66_merged.pdf"
all_text = ""

with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(tqdm(pdf.pages, desc="🔍 Extracting")):
        text = page.extract_text()

        if text and text.strip():
            extracted_text = text.strip()
            method = "Text"
        else:
            # OCR เฉพาะหน้านั้น
            pil_image = convert_from_path(pdf_path, dpi=500, first_page=i+1, last_page=i+1)[0]
            image_np = np.array(pil_image)

            # แปลงภาพเป็น grayscale
            gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
            gray = cv2.bilateralFilter(gray, 9, 75, 75)

            # Adaptive threshold เพื่อให้ภาพเป็นขาว-ดำ
            thresh = cv2.adaptiveThreshold(gray, 255,
                                          cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY_INV, 15, 9)  # ใช้ THRESH_BINARY_INV เพื่อหาเส้นดำบนพื้นขาว

            # ⛔ ลบเส้นแนวนอน แนวตั้ง
            horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
            remove_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
            vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
            remove_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
            mask = cv2.add(remove_horizontal, remove_vertical) # รวมเส้นที่ต้องลบ
            no_lines = cv2.bitwise_and(thresh, thresh, mask=cv2.bitwise_not(mask)) # ลบเส้นออกจากภาพต้นฉบับ
            final_image = cv2.bitwise_not(no_lines)# แปลงกลับเป็นพื้นขาว

            scale_percent = 150
            width = int(image_np.shape[1] * scale_percent / 100)
            height = int(image_np.shape[0] * scale_percent / 100)
            resized = cv2.resize(image_np, (width, height), interpolation=cv2.INTER_CUBIC)

            # OCR
            custom_config = '--oem 3 --psm 11 -l tha+eng'
            extracted_text = pytesseract.image_to_string(resized, config=custom_config)
            method = "OCR"

        print(f"\n--- Page {i+1} [{method}] ---\n{extracted_text}")
        all_text += f"\n\n--- Page {i+1} [{method}] ---\n{extracted_text}"

# บันทึกข้อความรวมทุกหน้า
with open("thai_pdf_output.txt", "w", encoding="utf-8") as f:
    f.write(all_text)


🔍 Extracting:  50%|█████     | 1/2 [00:00<00:00,  5.64it/s]


--- Page 1 [Text] ---
5.5 กรณีใช้ราคาที่เคยซื้อหรือจ้างครั้งหลังสุดภายในระยะเวลาสองปีงบประมาณให้ระบุว่าใช้ราคาที่เคย
ซื้อครั้งหลังสุดภายในระยะเวลา 2 ปีงบประมาณ พร้อมระบุเลขที่สัญญาที่ใช้อ้างอิง เช่น ตามสัญญาเลขที่ ...............
ลงวันที่ ....... เดือน ....................... พ.ศ. ................
ตัวอย่าง
ใช้ราคาที่เคยซื้อครั้งหลังสุดภายในระยะเวลา 2 ปีงบประมาณ ตามสัญญาเลขที่ ........ ลงวันที่ .......
5.6 กรณีใช้ราคาอื่นตามหลักเกณฑ์ วิธีการ หรือแนวทางปฏิบัติของ บมจ. โทรคมนาคมแห่งชาติ
ให้ระบุว่า ใช้ราคาตามหลักเกณฑ์ วิธีการ หรือแนวทางปฏิบัติ ก าหนด พร้อมทั้งระบุหลักเกณฑ์ที่หน่วยงานก าหนด
เช่น ตามข้อบังคับของหน่วยงาน ............. ลงวันที่ ................. เป็นต้น
หมายเหตุ
1. กรณีการเช่า และแลกเปลี่ยน ให้เจ้าหน้าที่ประกาศเปิดเผยข้อมูลรายละเอียดค่าใช้จ่ายการจัดซื้อจัดจ้าง
ราคากลาง และการค านวณราคากลาง ตามตารางนี้ด้วย
2. กรณีการซื้อที่ดิน หรือสิ่งก่อสร้าง ถ้าใช้ราคาประเมินของทางราชการหรือหน่วยงานเป็นราคาอ้างอิง
ให้ระบุชื่อหน่วยงานผู้ท าการประเมิน เช่น อ้างอิงตามราคาประเมิน... (ชื่อหน่วยงา

🔍 Extracting: 100%|██████████| 2/2 [00:43<00:00, 21.98s/it]


--- Page 2 [OCR] ---
(ฉบับประมวลครั้งที่ 1 ลงวันที่ 11 ธันวาคม 2567)

บริษัท โทรคมนาคมแหงชาติ จํากัด (มหาชน)

nt

National Telecom Public Company Limited

คําสังบริษัท โทรคมนาคมแห่งชาติ จํากัด (มหาชน)

ที่ รพ. 3 / 2567

เรื่อง วิธีปฏิบัติเกี่ยวกับการจัดซื้อจัดจ้างและการบริหารพัสดุที่เกี่ยวกับการพาณิชย์โดยตรง

เพื่อใหการดําเนินการจัดซื้อจัดจ่างและการบริหารพัสดุที่เกี่ยวกับการพาณิชย์โดยตรง เป็นไปด้วย

ความเรียบรอย โดยยึดหลักการความคุ่มค่า โปรงใส มีประสิทธิภาพและประสิทธิผล ตรวจสอบได้ และสอดคล้อง

กับระเบียบคณะกรรมการบริษัท โทรคมนาคมแหงชาติ จํากัด (มหาชน) ว่าด่วยการจัดซื้อจัดจางและการบริหาร

พัสดุที่เกี่ยวกับการพาณิชย์โดยตรง พ.ศ.2564

อาศัยอํานาจตามความในขอ 5 ของระเบียบคณะกรรมการบริษัท โทรคมนาคมแหงชาติ จํากัด

(มหาชน) ว่าด้วยการจัดซื้อจัดจ่างและการบริหารพัสดุที่เกี่ยวกับการพาณิชย์โดยตรง พ.ศ.2564 ประกอบกับ

ขอ 3 ของคําสังคณะกรรมการบริษัท โทรคมนาคมแห่งชาติ จํากัด (มหาชน) ที่ 20/2565 เรื่อง มอบอํานาจ

ใหกรรมการผูจัดการใหญ่ปฏิบัติการแทนคณะกรรมการบริษัท โทรคมนาคมแห่งชาติ จํากัด (มหาชน) ในการ



In [26]:
# Note ***
''' กรณีที่สไลด์นำเสนอเหมือนกัน สไลด์ขององค์กรมี templaetเดียวกัน'''
# Note ***

pdf_path = "/content/4_สายงานยุทธศาสตร์องค์กร_1 ก.พ.68.pptx.pdf"
all_text = ""

# พิกัด (x0, top, x1, bottom) ที่ต้องการครอป
crop_box = (225, 50, 960, 540)

with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        if i == 0:
            # ไม่ครอปหน้าแรก
            text = page.extract_text()
            print(f"\n📄 หน้า {i+1} (ไม่ครอป):\n{text}")
        elif i == 1:
            # ไม่ครอปหน้า2 เพราะเป็นเหมือนสารบัญและถ้าทำocr modelจะอ่านไม่เข้าใจ ซึ่งถ้าจะใช้ต้องทำการ โยงหรือทำ Chain-of-Thought (CoT)ก่อน
            text = ""
            print(f"\n📄 หน้า {i+1} (ข้าม)")
        else:
            # ครอปหน้าที่เหลือ
            cropped_page = page.within_bbox(crop_box)
            text = cropped_page.extract_text()
            print(f"\n📄 หน้า {i+1} (ครอป):\n{text}")
        all_text += f"\n--- Page {i+1}---\n{text}"

# บันทึกข้อความรวมทุกหน้า
with open("4_สายงานยุทธศาสตร์องค์กร_1 ก.พ.68.txt", "w", encoding="utf-8") as f:
    f.write(all_text)



📄 หน้า 1 (ไม่ครอป):
NT Org Chart @2024
สายงานยุทธศาสตรองคกร
วันที่ 1 กุมภาพันธ 2568
ปรับปรุงหนาที่ความรับผิดชอบของสวนงาน
(ตามระเบียบบริษัทฯ วาดวย โครงสรางบริษัทและการบริหารสวนงาน พ.ศ. 2567
ฉบับที่ 3 พ.ศ. 2568 ทั้งนี้ตั้งแตวันที่ 1 กุมภาพันธ 2568)

📄 หน้า 2 (ข้าม)

📄 หน้า 3 (ครอป):
สายงานยุทธศาสตรองคกร
1.
กำหนดนโยบาย กรอบ และแนวทางการดำเนินงานดานการวางแผนยุทธศาสตรองคกร การพัฒนาองคกรเพื่อความยั่งยืน การประเมินผลสวนงาน การบริหาร
การเปลี่ยนแปลง การบริหารความเสี่ยงและควบคุมภายใน นวัตกรรมองคกร การพัฒนาธุรกิจ และรัฐกิจสัมพันธ
2.
กำหนดนโยบาย กรอบ/ ทิศทาง กลยุทธ และ Roadmap การขับเคลื่อนไปสูองคกรดิจิทัล รวมถึง Enterprise Architecture ดาน Business Architecture
และ
ในภาพรวมองคกร
3.
กำหนดนโยบาย และแนวทางการดำเนินงานตามกรอบแนวทางการประเมินผลการดำเนินงานรัฐวิสาหกิจ ตามระบบ State Enterprise Assessment
Model (SE-AM) ดานการกำกับดูแลที่ดีและการนำองคกร (เนนดานการนำองคกรเชิงกลยุทธ) ดานการวางแผนเชิงกลยุทธ ดานการบริหารความเสี่ยงและ
ควบคุมภายใน และ
ดานการมุงเนนผูมีสวนไ

#Cleaning by Rule base & LLM as Judge

In [11]:
# For setup LLM as Judge
filepath = "4_สายงานยุทธศาสตร์องค์กร_1 ก.พ.68.txt"
# filepath = 'thai_pdf_output (1).txt'

# อ่านไฟล์ข้อความจาก OCR
def read_text_file(filepath: str) -> str:
    with open(filepath, 'r', encoding='utf-8') as f:
        return f.read()

all_text = read_text_file(filepath)

In [7]:
!pip install -q openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.8 MB/s eta 0:00:00


In [8]:
import openai
import re

# vLLM
openai.api_base = "http://1.179.140.78:8000/v1"
openai.api_key = "nta100_test_00"

def fix_text_with_llm(text: str) -> str:
    # ประมาณการขนาด
    input_length = len(text)

    if input_length < 1000:      # ข้อความสั้น
        max_tokens = 800
    elif input_length < 3000:    # ข้อความปานกลาง
        max_tokens = 1500
    elif input_length < 6000:    # ข้อความยาว
        max_tokens = 2500
    else:                        # ข้อความยาวมาก
        max_tokens = 4000

    prompt = f"""
ต่อไปนี้คือข้อความ OCR ที่อาจมีคำสะกดผิดหรือพิมพ์ผิดจากเอกสารราชการไทย โปรดแก้ไขเฉพาะคำที่ผิดให้ถูกต้อง ห้ามเพิ่มคำใหม่ หรือแก้ไขเนื้อหาที่ไม่ใช่คำผิด ห้ามอธิบายหรือแสดงความคิดเห็นใดๆ ห้ามแสดงกระบวนการคิด หรือแท็ก <think> และ </think> ตอบกลับเพียงข้อความที่แก้ไขแล้วเท่านั้น โดยไม่มีบรรทัดว่างหรือข้อความอื่นเพิ่มเติม

ข้อความ OCR: \"\"\"{text}\"\"\"
"""

    response = openai.ChatCompletion.create(
        model="Qwen/Qwen3-14B",
        messages=[
            {"role": "system", "content": "คุณคือผู้ช่วยที่แก้ไขข้อความ OCR อย่างเคร่งครัด ห้ามใช้ reasoning หรือแท็ก <think> ใดๆ ทั้งสิ้น"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0.1, enable_thinking=False
    )
    output = response['choices'][0]['message']['content'].strip()

    return output

def split_pages(text):
    parts = re.split(r'(--- Page \d+ \[(?:OCR|Text)\] ---)', text)
    pages = []
    for i in range(1, len(parts), 2):
        header = parts[i]
        content = parts[i + 1].strip()
        page_type = 'OCR' if '[OCR]' in header else 'Text'
        pages.append({'header': header, 'type': page_type, 'content': content})
    return pages

In [17]:
final_result = ""
for i, page in enumerate(split_pages(all_text), start=1):
    if page['type'] == 'OCR':
        Judge = fix_text_with_llm(page['content']) # LLM as Judge
        fixed = Judge.replace('<think>\n</think>\n\n"""', '').replace('"""', '') # Rule base

    else:
        fixed = page['content'].replace(" า", "ำ")  # Rule base

    final_result += f"--- Page {i} [{page['type']}] ---\n{fixed}\n\n"

print(final_result)

# Note ***
''' กรณีที่สไลด์นำเสนอเหมือนกัน สไลด์ขององค์กรมี templaetเดียวกัน'''

In [32]:
# Note ***
''' กรณีที่สไลด์นำเสนอเหมือนกัน สไลด์ขององค์กรมี templaetเดียวกัน'''
# Note ***

# ใช้ rule baseแก้ text ที่เป็นภาษาคอม
def replace_pua_chars(text):
    return (
        text.replace('\uf700', '่')  # ไม้เอก
            .replace('\uf701', '้')  # ไม้โท
            .replace('\uf702', '๊')  # ไม้ตรี
            .replace('\uf703', '๋')  # ไม้จัตวา
            .replace('\uf704', '็')  # ไม้ไต่คู้
            .replace('\uf712', '็')  # ไม้ไต่คู้
            .replace('\uf705', 'ั')  # ไม้หันอากาศ
            .replace('\uf706', 'ิ')  # สระอิ
            .replace('\uf707', 'ี')  # สระอี
            .replace('\uf708', 'ึ')  # สระอึ
            .replace('\uf709', 'ื')  # สระอื
            .replace('\uf70a', '่')  # (บางฟอนต์ = ไม้เอก)
            .replace('\uf70b', '้')  # (บางฟอนต์ = ไม้โท)
            .replace('\uf70c', '๊')
            .replace('\uf70d', '๋')
            .replace('\uf70e', '์')  # การันต์
            .replace('\uf70f', 'ํ')  # นฤคันต์
            .replace('\uf710', '๎')  # ยามักการ
    )

final_result = replace_pua_chars(all_text)
print(final_result)


--- Page 1---
NT Org Chart @2024
สายงานยุทธศาสตร์องค์กร
วันที่ 1 กุมภาพันธ์ 2568
ปรับปรุงหน้าที่ความรับผิดชอบของส่วนงาน
(ตามระเบียบบริษัทฯ ว่าด้วย โครงสร้างบริษัทและการบริหารส่วนงาน พ.ศ. 2567
ฉบับที่ 3 พ.ศ. 2568 ทั้งนี้ตั้งแต่วันที่ 1 กุมภาพันธ์ 2568)
--- Page 2---

--- Page 3---
สายงานยุทธศาสตร์องค์กร
1.
กำหนดนโยบาย กรอบ และแนวทางการดำเนินงานด้านการวางแผนยุทธศาสตร์องค์กร การพัฒนาองค์กรเพื่อความยั่งยืน การประเมินผลส่วนงาน การบริหาร
การเปลี่ยนแปลง การบริหารความเสี่ยงและควบคุมภายใน นวัตกรรมองค์กร การพัฒนาธุรกิจ และรัฐกิจสัมพันธ์
2.
กำหนดนโยบาย กรอบ/ ทิศทาง กลยุทธ์ และ Roadmap การขับเคลื่อนไปสู่องค์กรดิจิทัล รวมถึง Enterprise Architecture ด้าน Business Architecture
และ
ในภาพรวมองค์กร
3.
กำหนดนโยบาย และแนวทางการดำเนินงานตามกรอบแนวทางการประเมินผลการดำเนินงานรัฐวิสาหกิจ ตามระบบ State Enterprise Assessment
Model (SE-AM) ด้านการกำกับดูแลที่ดีและการนำองค์กร (เน้นด้านการนำองค์กรเชิงกลยุทธ์) ด้านการวางแผนเชิงกลยุทธ์ ด้านการบริหารความเสี่ยงและ
ควบคุมภายใน และ
ด้านการมุ่งเน้นผู้มีส่วนได้ส่วนเสียแล

In [33]:
# บันทึกข้อความรวมทุกหน้า
with open("result_ยุทธศาสตร์องค์กร.txt", "w", encoding="utf-8") as f:
    f.write(final_result)

#---------- Rag Part ----------

# Cleaing & Chunks

In [1]:
# For setup LLM as Judge
filepath = 'result_ยุทธศาสตร์องค์กร.txt'
# filepath = 'thai_pdf_output (1).txt'

# อ่านไฟล์ข้อความจาก OCR
def read_text_file(filepath: str) -> str:
    with open(filepath, 'r', encoding='utf-8') as f:
        return f.read()

final_result = read_text_file(filepath)

Cleaing & Chunks

In [36]:
# Note ***
''' old method'''
# Note ***

import re

def clean_text_for_rag(text: str) -> str:
    # ทำความสะอาดข้อความจากผลลัพธ์ LLM ให้เหมาะสำหรับ RAG

    # 1. ลบ thinking tags และเนื้อหาข้างใน
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    text = re.sub(r'<thinking>.*?</thinking>', '', text, flags=re.DOTALL)

    # 2. ลบ triple quotes และเนื้อหาข้างใน (ถ้าเป็น code blocks)
    text = re.sub(r'```.*?```', '', text, flags=re.DOTALL)
    text = re.sub(r'""".*?"""', '', text, flags=re.DOTALL)
    text = re.sub(r"'''.*?'''", '', text, flags=re.DOTALL)

    # 3. ลบ markdown headers และ formatting
    text = re.sub(r'^#{1,6}\s+', '', text, flags=re.MULTILINE)  # Headers
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)  # Bold
    text = re.sub(r'\*(.*?)\*', r'\1', text)  # Italic

    # 4. ลบ metadata และ system messages
    text = re.sub(r'^\[.*?\]', '', text, flags=re.MULTILINE)
    text = re.sub(r'^System:', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Assistant:', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Human:', '', text, flags=re.MULTILINE)

    # 5. ทำความสะอาด whitespace
    text = re.sub(r'\n\s*\n', '\n\n', text)  # ลด multiple newlines
    text = re.sub(r' +', ' ', text)  # ลด multiple spaces
    text = text.strip()

    text = re.sub(r'\s*\n\s*', ' ', text)  # ลบ \n และเว้นบรรทัดเกินความจำเป็น
    text = re.sub(r'\s+', ' ', text).strip()  # ลบช่องว่างซ้ำ ๆ

    # 6. ลบบรรทัดที่มีแค่เครื่องหมายพิเศษ
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        line = line.strip()
        # เก็บเฉพาะบรรทัดที่มีเนื้อหาจริง
        if line and not re.match(r'^[^\w\u0E00-\u0E7F]+$', line):
            cleaned_lines.append(line)

    return '\n'.join(cleaned_lines)

def clean_ocr_result_for_rag(ocr_result: str) -> str:
    """
    ทำความสะอาดผลลัพธ์ OCR ทั้งหมดให้เหมาะสำหรับ RAG
    """
    pages = []

    # แยกตาม page headers
    parts = re.split(r'(--- Page \d+ \[(?:OCR|Text)\] ---)', ocr_result)

    for i in range(1, len(parts), 2):
        if i + 1 < len(parts):
            header = parts[i]
            content = parts[i + 1]

            # ทำความสะอาดเนื้อหา
            cleaned_content = clean_text_for_rag(content)

            if cleaned_content:
                # Extract page number
                page_match = re.search(r'Page (\d+)', header)
                page_num = page_match.group(1) if page_match else str(len(pages) + 1)

                pages.append({
                    'page': int(page_num),
                    'content': cleaned_content
                })

    return pages

def prepare_text_for_chunking(cleaned_pages: list, chunk_size: int = 1000) -> list:
    """
    เตรียมข้อความสำหรับการแบ่ง chunks สำหรับ RAG
    """
    chunks = []

    for page_data in cleaned_pages:
        page_num = page_data['page']
        content = page_data['content']

        # แบ่งเป็น sentences
        sentences = re.split(r'[.!?]+', content)

        current_chunk = ""
        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            # ถ้าเพิ่ม sentence นี้แล้วยาวเกินไป
            if len(current_chunk + sentence) > chunk_size:
                if current_chunk:
                    chunks.append({
                        'text': current_chunk.strip(),
                        'page': page_num,
                        'chunk_id': len(chunks) + 1
                    })
                    current_chunk = sentence
                else:
                    # ถ้า sentence เดียวยาวเกิน chunk_size
                    chunks.append({
                        'text': sentence,
                        'page': page_num,
                        'chunk_id': len(chunks) + 1
                    })
            else:
                current_chunk += " " + sentence if current_chunk else sentence

        # เพิ่ม chunk สุดท้าย
        if current_chunk:
            chunks.append({
                'text': current_chunk.strip(),
                'page': page_num,
                'chunk_id': len(chunks) + 1
            })

    return chunks

def process_ocr_for_rag(final_result: str) -> dict:
    """
    Process OCR result และเตรียมสำหรับ RAG system
    """
    # 1. ทำความสะอาด
    cleaned_pages = clean_ocr_result_for_rag(final_result)

    # 2. เตรียม chunks
    chunks = prepare_text_for_chunking(cleaned_pages)

    # 3. สร้าง metadata
    metadata = {
        'total_pages': len(cleaned_pages),
        'total_chunks': len(chunks),
        'avg_chunk_length': sum(len(chunk['text']) for chunk in chunks) / len(chunks) if chunks else 0
    }

    return {
        'pages': cleaned_pages,
        'chunks': chunks,
        'metadata': metadata
    }

# ตัวอย่างการใช้
if __name__ == "__main__":
    # สมมติว่า final_result คือผลลัพธ์จาก OCR correction
    processed_data = process_ocr_for_rag(final_result)

    print(f"จำนวนหน้า: {processed_data['metadata']['total_pages']}")
    print(f"จำนวน chunks: {processed_data['metadata']['total_chunks']}")
    print(f"ความยาวเฉลี่ย: {processed_data['metadata']['avg_chunk_length']:.0f} ตัวอักษร")

    # แสดงตัวอย่าง chunks
    for i, chunk in enumerate(processed_data['chunks'][:3]):
        print(f"\nChunk {i+1} (Page {chunk['page']}):")
        print(chunk['text'][:200] + "..." if len(chunk['text']) > 200 else chunk['text'])

จำนวนหน้า: 0
จำนวน chunks: 0
ความยาวเฉลี่ย: 0 ตัวอักษร


# Note ***
''' กรณีที่สไลด์นำเสนอเหมือนกัน สไลด์ขององค์กรมี templaetเดียวกัน'''

**ลองใช้วิธีใหม้ในการแบ่ง chunk โดยใช้ libraryจาก thainlpมาแบ่งย่อหน้า**

In [3]:
!pip install -q pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 74.9 MB/s eta 0:00:00


In [13]:
# Note ***
''' กรณีที่สไลด์นำเสนอเหมือนกัน สไลด์ขององค์กรมี templaetเดียวกัน'''
# Note ***

import re
from pythainlp.tokenize import sent_tokenize  # ใช้ของ pythainlp

def process_ocr_for_rag(text, max_chunk_size=500):
    import re
    from pythainlp.tokenize import sent_tokenize

    pages = re.split(r'\n--- Page \d+---\n', text)
    pages = [p.strip() for p in pages if p.strip()]

    chunks = []
    chunk_counter = 1 # Initialize chunk counter
    for page_idx, page_text in enumerate(pages, start=1):
        sentences = sent_tokenize(page_text, engine="whitespace")
        current_chunk = ""
        for sentence in sentences:
            if len(current_chunk) + len(sentence) <= max_chunk_size:
                current_chunk += sentence + " "
            else:
                chunks.append({"page": page_idx, "text": current_chunk.strip(), "chunk_id": chunk_counter}) # Add chunk_id
                chunk_counter += 1
                current_chunk = sentence + " "
        if current_chunk.strip():
            chunks.append({"page": page_idx, "text": current_chunk.strip(), "chunk_id": chunk_counter}) # Add chunk_id
            chunk_counter += 1

    total_pages = len(pages)
    total_chunks = len(chunks)
    avg_chunk_length = sum(len(c['text']) for c in chunks) / total_chunks if total_chunks else 0

    return {
        "metadata": {
            "total_pages": total_pages,
            "total_chunks": total_chunks,
            "avg_chunk_length": avg_chunk_length,
        },
        "chunks": chunks,
    }


if __name__ == "__main__":
    # สมมติว่า final_result คือผลลัพธ์จาก OCR correction
    processed_data = process_ocr_for_rag(final_result)

    print(f"จำนวนหน้า: {processed_data['metadata']['total_pages']}")
    print(f"จำนวน chunks: {processed_data['metadata']['total_chunks']}")
    print(f"ความยาวเฉลี่ย: {processed_data['metadata']['avg_chunk_length']:.0f} ตัวอักษร")

    # แสดงตัวอย่าง chunks
    for i, chunk in enumerate(processed_data['chunks'][:3]):
        print(f"\nChunk {i+1} (Page {chunk['page']}):")
        print(chunk['text'][:200] + "..." if len(chunk['text']) > 200 else chunk['text'])

จำนวนหน้า: 40
จำนวน chunks: 161
ความยาวเฉลี่ย: 412 ตัวอักษร

Chunk 1 (Page 1):
NT Org Chart @2024
สายงานยุทธศาสตร์องค์กร
วันที่ 1 กุมภาพันธ์ 2568
ปรับปรุงหน้าที่ความรับผิดชอบของส่วนงาน
(ตามระเบียบบริษัทฯ ว่าด้วย โครงสร้างบริษัทและการบริหารส่วนงาน พ.ศ. 2567
ฉบับที่ 3 พ.ศ. 2568 ทั...

Chunk 2 (Page 2):
สายงานยุทธศาสตร์องค์กร
1.
กำหนดนโยบาย กรอบ และแนวทางการดำเนินงานด้านการวางแผนยุทธศาสตร์องค์กร การพัฒนาองค์กรเพื่อความยั่งยืน การประเมินผลส่วนงาน การบริหาร
การเปลี่ยนแปลง การบริหารความเสี่ยงและควบคุมภา...

Chunk 3 (Page 2):
ตามระบบ State Enterprise Assessment
Model (SE-AM) ด้านการกำกับดูแลที่ดีและการนำองค์กร (เน้นด้านการนำองค์กรเชิงกลยุทธ์) ด้านการวางแผนเชิงกลยุทธ์ ด้านการบริหารความเสี่ยงและ
ควบคุมภายใน และ
ด้านการมุ่งเน...


#Embedding

In [6]:
!pip install -q transformers sentencepiece torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 984.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.5 MB/s eta 0:00:00


In [14]:
from transformers import AutoTokenizer, AutoModel
import torch
from textwrap import wrap
import numpy as np

# โหลด tokenizer และ model
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
model = AutoModel.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")

MAX_CHAR_LEN = 600  # ห้ามเกินเพื่อไม่ให้ token เยอะเกิน 512

def get_thai_embedding(text):
    try:
        inputs = tokenizer(
            text, return_tensors="pt", padding=True, truncation=True, max_length=512
        )
        if inputs["input_ids"].shape[1] == 0:
            print(f"⚠️ No tokens found in text: {repr(text[:50])}")
            return None
        with torch.no_grad():
            outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    except Exception as e:
        print(f"🚨 Error embedding text: {repr(text[:50])} -> {e}")
        return None

In [15]:
from tqdm import tqdm
embeddings_data = []

for chunk in tqdm(processed_data['chunks']):
    text = chunk["text"].strip()
    if not text:
        continue

    # ตัดข้อความยาวเกิน MAX_CHAR_LEN ออกเป็นหลายชิ้น
    parts = wrap(text, MAX_CHAR_LEN)

    part_embeddings = []
    for part in parts:
        emb = get_thai_embedding(part)
        if emb is not None:
            part_embeddings.append(emb)

    if part_embeddings:
        # รวม embedding ของทุกส่วนด้วย mean หรือ concat ตามที่เหมาะสม
        combined_embedding = np.mean(part_embeddings, axis=0)
        embeddings_data.append({
            "page": chunk["page"],
            "chunk_id": chunk["chunk_id"], # Use the correct key 'chunk_id'
            "text": text, # Add the original text here
            "embedding": combined_embedding
        })

  0%|          | 0/161 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `CamembertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 161/161 [01:14<00:00,  2.16it/s]


In [17]:
embeddings_data

[{'page': 1,
  'chunk_id': 1,
  'text': 'NT Org Chart @2024\nสายงานยุทธศาสตร์องค์กร\nวันที่ 1 กุมภาพันธ์ 2568\nปรับปรุงหน้าที่ความรับผิดชอบของส่วนงาน\n(ตามระเบียบบริษัทฯ ว่าด้วย โครงสร้างบริษัทและการบริหารส่วนงาน พ.ศ. 2567\nฉบับที่ 3 พ.ศ. 2568 ทั้งนี้ตั้งแต่วันที่ 1 กุมภาพันธ์ 2568)',
  'embedding': array([ 0.54169136, -0.83307767,  0.4922719 ,  0.21086861, -0.20183998,
          0.20911781,  1.1290219 , -0.19047447, -0.1307865 , -0.6709386 ,
          0.2749254 ,  0.50744826,  0.15837663,  0.2289169 , -0.74920005,
         -1.4598771 , -0.5140648 ,  0.15761214,  0.08129779, -0.22622383,
          0.09562479, -0.2174126 , -0.12804204, -0.19208856, -0.19320573,
         -0.41841218, -0.17498408,  0.3570799 , -0.15466763,  0.7584111 ,
         -0.11704494, -0.23711678,  0.31681362,  0.11208376, -0.41789475,
          1.581239  , -0.9982411 , -0.08652519,  0.32050636,  0.95110905,
          0.57459927, -0.02016398, -0.21961418,  0.1388641 , -0.53901607,
          0.36275733, -0.85945356, 

In [20]:
import pickle

# เซฟ
with open('embeddings_data.pkl', 'wb') as f:
    pickle.dump(embeddings_data, f)

# FAISS

**FAISS**

เก็บ embeddings เหล่านั้นไว้ในฐานข้อมูล vector store

In [2]:
import pickle

# โหลด
with open('embeddings_data.pkl', 'rb') as f:
    embeddings_data  = pickle.load(f)

In [1]:
!pip install -q FAISS-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.4 MB/s eta 0:00:00


In [24]:
import faiss
import numpy as np

# เตรียม numpy array ของ vectors เท่านั้น (สมมติ embedding dimension เท่ากันทั้งหมด)
vectors = np.array([item['embedding'] for item in embeddings_data]).astype('float32')

# สร้าง FAISS index แบบ Flat (simple, exact search)
dimension = vectors.shape[1]  # ขนาด embedding dimension
index = faiss.IndexFlatL2(dimension)  # ใช้ระยะห่างแบบ L2 (Euclidean)

# เพิ่ม vectors เข้า index
index.add(vectors)

# ทดสอบค้นหา
query_vector = vectors[0].reshape(1, -1)  # สมมติใช้ vector ตัวแรกค้นหา
k = 5  # จำนวนผลลัพธ์ที่ต้องการ

distances, indices = index.search(query_vector, k)
print("Nearest neighbors indices:", indices)
print("Distances:", distances)

# ดึง metadata ที่เกี่ยวข้อง (page, chunk_id) จาก indices
for idx in indices[0]:
    meta = embeddings_data[idx]
    print(f"Page: {meta['page']}, Chunk ID: {meta['chunk_id']}")


Nearest neighbors indices: [[  0  39  75 106  83]]
Distances: [[ 0.       73.763336 74.451866 76.57535  76.93681 ]]
Page: 1, Chunk ID: 1
Page: 12, Chunk ID: 40
Page: 20, Chunk ID: 76
Page: 27, Chunk ID: 107
Page: 22, Chunk ID: 84


แปลงคำถามเป็น embedding ด้วยโมเดลเดียวกันหรือใกล้เคียง

ดึงข้อมูลที่ใกล้เคียงจาก vector store มาเป็น context (retrieval step)

In [5]:
# เรียกโมเดล embeddingอีกครั้ง
from sentence_transformers import SentenceTransformer
import numpy as np

# โหลดโมเดล embedding ภาษาไทย
embed_model = SentenceTransformer("airesearch/wangchanberta-base-att-spm-uncased")

def get_thai_embedding(text: str) -> np.ndarray:
    embedding = embed_model.encode([text])[0]
    return np.array(embedding).astype('float32')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

In [25]:
def ask_rag(query_text: str, index, embeddings_data, k):
    # 1. Embed คำถาม
    query_embedding = get_thai_embedding(query_text).reshape(1, -1)

    # 2. ดึง context จาก FAISS
    distances, indices = index.search(query_embedding, k=k)
    retrieved_contexts = [embeddings_data[i]['text'] for i in indices[0]]

    # 3. รวมเป็น prompt
    context = "\n\n".join(retrieved_contexts)
    prompt = f"""ข้อมูลที่เกี่ยวข้อง:
{context}\n\n
คำถาม: {query_text}
กรุณาตอบคำถามจากข้อมูลข้างต้นอย่างชัดเจนและกระชับ"""

    return prompt

# generate

**ส่ง context ที่ดึงมา พร้อมกับคำถาม ไปยังโมเดลภาษาที่ใช้สร้างคำตอบ**

โมเดลจะ “generate” คำตอบโดยอิงข้อมูลจาก context


In [7]:
!pip install -q openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.8 MB/s eta 0:00:00


In [8]:
import openai
import requests
import re

# vLLM
openai.api_base = "http://1.179.140.78:8000/v1"
openai.api_key = "nta100_test_00"

def generate_answer_with_vllm(prompt: str):
    # ประมาณการขนาด
    input_length = len(prompt)

    if input_length < 1000:      # ข้อความสั้น
        max_tokens = 800
    elif input_length < 3000:    # ข้อความปานกลาง
        max_tokens = 1500
    elif input_length < 6000:    # ข้อความยาว
        max_tokens = 2500
    else:                        # ข้อความยาวมาก
        max_tokens = 4000

    response = openai.ChatCompletion.create(
        model="Qwen/Qwen3-14B",
        messages=[
            {"role": "system", "content": "คุณคือตัวช่วยตอบคำถามจากข้อมูล"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0.7,
        enable_thinking=False
    )

    raw_output = response['choices'][0]['message']['content']
    answer = re.sub(r"<think>.*?</think>", "", raw_output, flags=re.DOTALL).strip() # clean_response
    return answer

In [28]:
# คำถาม
query_text = "กระบวนการของกลุ่มยุทธศาสตร์"

# RAG: Retrieve + Prompt
prompt = ask_rag(query_text, index, embeddings_data, k=5)

# Generate answer ด้วย vLLM
answer = generate_answer_with_vllm(prompt)
print("คำตอบ:", answer)

คำตอบ: กระบวนการของกลุ่มยุทธศาสตร์ (ตามข้อมูลที่ให้มา) แบ่งออกเป็น 2 ด้านหลัก ดังนี้:

### 1. **กลุ่มบริหารความต่อเนื่องทางธุรกิจ (BCM)**
- **ศึกษาและปรับใช้มาตรฐานสากล** (เช่น ISO 22301) และแนวทางปฏิบัติที่ดี  
- **จัดทำนโยบายและแผนปฏิบัติการ** ด้าน BCM อย่างเป็นระบบ  
- **ออกแบบแผน BCP และ BCM** ร่วมกับการประเมินความเสี่ยง (RA) และวิเคราะห์ผลกระทบทางธุรกิจ (BIA)  
- **ประสานงานกับหน่วยงานที่เกี่ยวข้อง** เพื่อสร้างความต่อเนื่องในองค์กร  
- **จัดทำแผนบริหารความเสี่ยง** เพื่อเพิ่มประสิทธิภาพการปฏิบัติงาน  

### 2. **กลุ่มโครงการนวัตกรรม**
- **ออกแบบและกำกับดูแล Sandbox** สำหรับทดสอบนวัตกรรมในช่วง POC (Proof of Concept)  
- **พิจารณางบประมาณ** และติดตามผลการดำเนินงานให้สอดคล้องกับเป้าหมายและกรอบเวลา  
- **ประเมินและรายงานผลนวัตกรรม** ให้ผู้บริหารทราบเป็นระยะ  
- **ดำเนินการวิจัย/ทดลอง/จัดกิจกรรม** เพื่อพัฒนานวัตกรรมอย่างเป็นรูปธรรม  
- **สนับสนุนการยกระดับมาตรฐานบริการ** ผ่านการใช้ข้อมูลเชิงวิชาการ (เช่น ศูนย์บริการลูกค้า)  

### 3. **การประเมินผลแบบองค์รวม**
- ดำเนินงานตาม **SE-AM (Stat

In [29]:
# คำถาม
query_text = "กลุ่มยุทธศาสตร์"

# RAG: Retrieve + Prompt
prompt = ask_rag(query_text, index, embeddings_data, k=5)

# Generate answer ด้วย vLLM
answer = generate_answer_with_vllm(prompt)
print("คำตอบ:", answer)

คำตอบ: **กลุ่มยุทธศาสตร์หลักจากข้อมูล:**  
1. **การบริหารจัดการเชิงยุทธศาสตร์และผู้มีส่วนได้ส่วนเสีย**  
   - ดำเนินการตามกรอบ SE-AM เน้นผู้มีส่วนได้ส่วนเสียและลูกค้า  
   - ศึกษาวิเคราะห์ผู้มีส่วนได้ส่วนเสีย ออกแบบระบบบริหารจัดการผู้มีส่วนได้ส่วนเสีย  
   - สนับสนุนศูนย์บริการลูกค้า (GECC) และปรับปรุงมาตรฐานการให้บริการ  

2. **การบริหารความเสี่ยงและแผนดำเนินงานต่อเนื่อง (BCM/BCP)**  
   - จัดทำและประเมินแผน BCM/BCP  
   - ติดตามความก้าวหน้า ประสานงานกับทีมตอบสนองสถานการณ์ฉุกเฉิน (IRT)  
   - สรุปผลการดำเนินงานและปรับปรุงแผน  

3. **การส่งเสริมนวัตกรรมและทรัพย์สินทางปัญญา**  
   - สนับสนุนนวัตกรรมผ่านความร่วมมือทางวิชาการ/ทุนวิจัย  
   - สร้างวัฒนธรรมความคิดสร้างสรรค์ในองค์กร  
   - ประเมินการจัดการนวัตกรรมเพื่อความยั่งยืน  

4. **การบริหารจัดการทรัพย์สิน**  
   - กำหนดหลักเกณฑ์การดูแลบำรุงรักษาอสังหาริมทรัพย์ (ที่ดิน อาคาร)  
   - ใช้ทรัพย์สินเพื่อสร้างรายได้และประโยชน์สูงสุด  

5. **การพัฒนาประสิทธิภาพองค์กร**  
   - ประเมินผลการดำเนินงานรัฐวิสาหกิจ  
   - สร้างความเชื่อมั่นในการดำเ